In [1]:
!pip3 install transformers
!pip3 install torch torchvision
!pip3 install datasets
!pip3 install tqdm
!pip3 install ipywidgets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 112.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.

In [17]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [47]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

# Load the model you trained
model = BertForQuestionAnswering.from_pretrained('nlpaueb/legal-bert-small-uncased')
state_dict = torch.load('/content/gdrive/My Drive/saved_model/model_LegalBert_QA_new.pth')
model.load_state_dict(state_dict)
model.eval()

# Load the corresponding tokenizer
tokenizer = BertTokenizer.from_pretrained('nlpaueb/legal-bert-small-uncased')

# Here's a sample paragraph and a question
#context = "The Supreme Court of the United States (SCOTUS) is the highest court in the federal judiciary of the United States of America. It has ultimate and largely discretionary appellate jurisdiction over all federal and state court cases"
#question = "What is SCOTUS?" 
#question = "Is SCOTUS a federal or state court?" 
#context = "There are two types of contract breaches recognized by the courts: material breach and immaterial breach. A material breach is a party's failure to perform a major part of the contract, which results in damaging the other party. An immaterial breach, on the other hand, is a minor and unimportant failure that does not damage the other party."
#question = "What is an immaterial breach of contract?"
#context="Intellectual property law grants owners exclusive rights to the products of human intellect, including patents, trademarks, and copyrights. A patent grants an inventor the right to exclude others from making, using, selling, offering to sell, or importing an invention for a certain number of years. "
#question = "What rights are granted by intellectual property law?"
#question = "What does copyright law protect?"

#context = "The amendment addresses citizenship rights and equal protection under the law and was proposed in response to issues related to former slaves following the American Civil War. The amendment provides a broad definition of citizenship, overruling the Supreme Court's decision in Dred Scott v. Sandford (1857)" 
#"The amendment provides a broad definition of citizenship, overruling the Supreme Court's decision in Dred Scott v. Sandford (1857), which had held that Black people were not, and could not become, citizens of the United States. "
#question = "When was the Fourteenth Amendment to the United States Constitution adopted?"
#question = "What does the Fourteenth Amendment address?"

context = "Employment law is a broad area that encompasses all areas of the employer/employee relationship. It includes not only collective labor legislation, which relates to the rights and duties of unions, employees, and employers, but also individual labor law, which concerns employees' rights at work and through the contract for work. Employment laws were first enacted to counter the harsh conditions that had become commonplace in factories and mines and to ensure fair pay and reasonable hours."
question = "What does employment law encompass?"
#"Examples might include petty theft, disorderly conduct, or trespassing. Felonies, on the other hand, include serious crimes like murder, rape, and burglary, and the punishments are typically more severe, often resulting in imprisonment for more than a year."

# We encode the context and question using the tokenizer
inputs = tokenizer.encode_plus(question, context, return_tensors='pt')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)
inputs = {key: value.to(device) for key, value in inputs.items()}

outputs = model(**inputs)

# We take the maximum scoring tokens as our predicted start and end tokens
answer_start = torch.argmax(outputs.start_logits) 
answer_end = torch.argmax(outputs.end_logits)

# And finally, convert these tokens back into string using the tokenizer
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end+1]))

print(answer)

Some weights of the model checkpoint at nlpaueb/legal-bert-small-uncased were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized f

collective labor legislation , which relates to the rights and duties of unions , employees , and employers , but also individual labor law , which


In [5]:
# Here's a sample paragraph and a question
#context = "The Supreme Court of the United States (SCOTUS) is the highest court in the federal judiciary of the United States of America. It has ultimate and largely discretionary appellate jurisdiction over all federal and state court cases that involve a point of federal law, and original jurisdiction over a narrow range of cases, specifically all Cases affecting Ambassadors, other public Ministers and Consuls, and those in which a State shall be Party. The Court holds the power of judicial review, the ability to invalidate a statute for violating a provision of the Constitution."
#question = "What does SCOTUS stand for?"
context = "In contract law, a party's failure to fulfill an end of the bargain under a contract is known as a breach of contract. When such a breach occurs, one or both of the parties may wish to have the contract enforced on its terms, or may try to recover for any financial harm caused by the alleged breach. There are two types of contract breaches recognized by the courts: material breach and immaterial breach. A material breach is a party's failure to perform a major part of the contract, which results in damaging the other party. An immaterial breach, on the other hand, is a minor and unimportant failure that does not damage the other party."
question = "What is a breach of contract in contract law?"
# We encode the context and question using the tokenizer
inputs = tokenizer.encode_plus(question, context, return_tensors='pt')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)
inputs = {key: value.to(device) for key, value in inputs.items()}

outputs = model(**inputs)

answer_start = torch.argmax(outputs.start_logits) 
answer_end = torch.argmax(outputs.end_logits) 

# And finally, convert these tokens back into string using the tokenizer
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end+1]))

print(answer)